In [2]:
!pip install keras_preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00


In [3]:
import numpy as np
from keras_preprocessing.sequence import pad_sequences
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
#from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.datasets import imdb


In [4]:
seed = 7
print(seed)

7


In [5]:
from sklearn.model_selection import train_test_split
from keras.datasets import imdb

# Загрузка данных
max_features = 20000
(x_data1, y_data1), (x_data2, y_data2) = imdb.load_data(num_words=max_features,seed=seed)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [6]:
max_review_length = 2000
x_data = np.concatenate([x_data1, x_data2], axis=0)
y_data = np.concatenate([y_data1, y_data2], axis=0)
x_data = pad_sequences(x_data, maxlen=max_review_length,value = 0,padding = 'post', truncating='post')

In [7]:

x_train, x_temp, y_train, y_temp = train_test_split(x_data, y_data, test_size=0.15, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.2, random_state=42)

# Проверка размеров выборок
print(x_train.shape, x_val.shape, x_test.shape)

(42500, 2000) (6000, 2000) (1500, 2000)


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

embedding_vecor_length = 300
# Модель
model_cnn = Sequential([
    Embedding(input_dim=max_features, output_dim=embedding_vecor_length),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_cnn.summary()
# Обучение
print(type(x_train[0][0]))
history_cnn  = model_cnn.fit(
    x_train, y_train,
    epochs=10,
    batch_size=128,verbose =1,validation_data=(x_val,y_val)
)


In [10]:
import os
import urllib.request
import zipfile

# Папка для хранения GloVe
glove_dir = './glove'

if not os.path.exists(glove_dir):
    os.makedirs(glove_dir)

# URL для скачивания GloVe
glove_url = 'http://nlp.stanford.edu/data/glove.6B.zip'
glove_zip_path = os.path.join(glove_dir, 'glove.6B.zip')

# Скачивание
if not os.path.exists(glove_zip_path):
    print("Скачивание GloVe...")
    urllib.request.urlretrieve(glove_url, glove_zip_path)

# Распаковка
if not os.path.exists(os.path.join(glove_dir, 'glove.6B.300d.txt')):
    print("Распаковка GloVe...")
    with zipfile.ZipFile(glove_zip_path, 'r') as zip_ref:
        zip_ref.extractall(glove_dir)

print("GloVe готов к использованию.")


Скачивание GloVe...
Распаковка GloVe...
GloVe готов к использованию.


In [13]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Dropout
import numpy as np

# Загрузка GloVe
embedding_dim = 300
glove_file = './glove/glove.6B.300d.txt'
embeddings_index = {}

with open(glove_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

glove.6B.100d.txt  glove.6B.200d.txt  glove.6B.300d.txt  glove.6B.50d.txt  glove.6B.zip


In [15]:
# Создание матрицы эмбеддингов
word_index = imdb.get_word_index()
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [16]:
# Построение полносвязной сети
model_dense = Sequential([
    Embedding(input_dim=max_features,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              input_length=max_review_length,
              trainable=False),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Компиляция модели
model_dense.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Обучение модели
history_dense = model_dense.fit(
    x_train, y_train,
    epochs=10,
    batch_size=128,
    validation_data=(x_val, y_val),
    verbose=1
)

Epoch 1/10
146/333 ━━━━━━━━━━━━━━━━━━━━ 9:44 3s/step - accuracy: 0.5066 - loss: 1.1333

In [ ]:
# Построение графиков
import matplotlib.pyplot as plt

def plot_history(history, title):
    plt.figure(figsize=(12, 5))
    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title(f'{title} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{title} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

# Визуализация
plot_history(history_dense, 'Dense Model')

# Оценка моделей

In [ ]:
y_pred_cnn = (model_cnn.predict(x_test) > 0.5).astype(int)

In [ ]:
from sklearn.metrics import classification_report, f1_score
import matplotlib.pyplot as plt
print("CNN Classification Report:")
print(classification_report(y_test, y_pred_cnn))
plt.plot(history_cnn.history['accuracy'], label='CNN Train Accuracy')
plt.plot(history_cnn.history['val_accuracy'], label='CNN Val Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.show()